# Creacion de un modelo de regresion logistica simple

## Data visualization

Comencemos este ejercicio cargando y examinando nuestros datos:

In [ ]:
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/avalanche.csv

#Import the data from the .csv file
dataset = pandas.read_csv('avalanche.csv', delimiter="\t")

#Let's have a look at the data
dataset

# Exploración de datos

El campo de avalanchas es nuestro objetivo. Un valor de 1 significa que ocurrió una avalancha en las condiciones descritas por las características, mientras que un valor de 0 significa que no ocurrió ninguna avalancha. Dado que nuestros objetivos solo pueden ser 0 o 1, llamamos a esto un modelo de clasificación binaria.

Ahora, vamos a graficar las relaciones entre cada característica y los valores objetivo. Eso nos ayuda a comprender qué características son más propensas a influir en los resultados:

In [ ]:
import graphing # custom graphing code. See our GitHub repo for details

graphing.box_and_whisker(dataset, label_x="avalanche", label_y="surface_hoar", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="fresh_thickness", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="weak_layers", show=True)
graphing.box_and_whisker(dataset, label_x="avalanche", label_y="no_visitors")

## Podemos observar que:

* Para fresh_thickness, los resultados son muy similares. Esto significa que las variaciones en sus valores no están fuertemente correlacionadas con los resultados.

* Las variaciones en los valores de weak_layers y no_visitors parecen estar correlacionadas con un mayor número de resultados de avalanchas, por lo que debemos asignar más importancia a estas características.

Las diferencias entre los días con avalanchas y los días sin avalanchas son pequeñas y no hay un factor claro que las explique. Weak layers parece ser un buen punto de partida ya que está relacionado con la mayor variación en los resultados.

## Construyendo un modelo simple de regresión logística

Ahora construiremos y entrenaremos un modelo para predecir la probabilidad de que ocurra una avalancha basándonos únicamente en el número de capas débiles de nieve:

In [ ]:
# Here we import a function that splits datasets according to a given ratio
from sklearn.model_selection import train_test_split

# Split the dataset in an 70/30 train/test ratio. 
train, test = train_test_split(dataset, test_size=0.3, random_state=2)
print(train.shape)
print(test.shape)

Acontinuación entrenemos nuestro modelo, usando el conjunto de datos de entrenamiento que acabamos de crear (ten en cuenta que weak_layers será la única característica utilizada para determinar el resultado):

In [ ]:
import statsmodels.formula.api as smf
import graphing # custom graphing code. See our GitHub repo for details

# Perform logistic regression.
model = smf.logit("avalanche ~ weak_layers", train).fit()

print("Model trained")

Después de entrenar, podemos imprimir un resumen del modelo con información muy detallada:

In [ ]:
print(model.summary())

Observa que el coeficiente positivo para weak_layers significa que un valor más alto de esta variable se traduce en una mayor probabilidad de que se produzca una avalancha.

# Usando nuestro modelo

Ahora podemos utilizar nuestro modelo entrenado para hacer predicciones y estimar probabilidades.

Tomemos las primeras cuatro ocurrencias en nuestro conjunto de prueba e imprimamos la probabilidad de una avalancha para cada una de ellas:

In [ ]:
# predict to get a probability

# get first 3 samples from dataset
samples = test["weak_layers"][:4]

# use the model to get predictions as possibilities
estimated_probabilities = model.predict(samples)

# Print results for each sample
for sample, pred in zip(samples,estimated_probabilities):
    print(f"A weak_layer with value {sample} yields a {pred * 100:.2f}% chance of an avalanche.")

Grafiquemos nuestro modelo para entender esto:

In [ ]:
# plot the model
# Show a graph of the result
predict = lambda x: model.predict(pandas.DataFrame({"weak_layers": x}))

graphing.line_2D([("Model", predict)],
                 x_range=[-20,40],
                 label_x="weak_layers", 
                 label_y="estimated probability of an avalanche")

La línea traza la función de la probabilidad de una avalancha en función del número de capas débiles; observa que cuanto más capas débiles hay, más probable es que ocurra una avalancha. Este gráfico puede parecer un poco confuso por dos razones.

En primer lugar, la curva puede hacer predicciones desde menos hasta más infinito, pero solo tenemos datos para 0 a 10 capas:

In [ ]:
print("Minimum number of weak layers:", min(train.weak_layers))
print("Maximum number of weak layers:", max(train.weak_layers))

Esto se debe a que los modelos de regresión logística permiten hacer predicciones fuera del rango de valores que han visto, y a veces lo hacen bastante bien.

La segunda razón por la que el gráfico es confuso es que con 0 capas todavía hay cierto riesgo de avalancha. De manera similar, con 10 capas no hay un riesgo del 100% de avalancha. De hecho, esto está en línea con los datos:

In [ ]:
import numpy as np

# Get actual rates of avalanches at 0 years
avalanche_outcomes_for_0_layers = train[train.weak_layers == 0].avalanche
print("Average rate of avalanches for 0 weak layers of snow", np.average(avalanche_outcomes_for_0_layers))

# Get actual rates of avalanches at 10 years
avalanche_outcomes_for_10_layers = train[train.weak_layers == 10].avalanche
print("Average rate of avalanches for 10 weak layers of snow", np.average(avalanche_outcomes_for_10_layers))

¡Nuestro modelo en realidad está haciendo un buen trabajo! Es solo que las avalanchas no son causadas únicamente por capas débiles de nieve. Si queremos mejorar, probablemente necesitamos pensar en incluir otra información en el modelo.


# La clasificación o los umbrales de decisión

Se refieren al valor que se utiliza para determinar si una predicción se considera positiva o negativa. Para devolver una categoría binaria (Verdadero = "avalancha", Falso = "sin avalancha"), necesitamos definir un valor de umbral de clasificación. Cualquier probabilidad por encima de ese umbral se devolverá como la categoría positiva, mientras que los valores por debajo se devolverán como la categoría negativa.

Veamos qué sucede si establecemos nuestro umbral en 0,5 (lo que significa que nuestro modelo devolverá Verdadero siempre que calcule una probabilidad superior al 50% de que ocurra una avalancha):

In [ ]:
# threshold to get an absolute value
threshold = 0.5

# Add classification to the samples we used before
for sample, pred in list(zip(samples,estimated_probabilities)):
    print(f"A weak_layer with value {sample} yields a chance of {pred * 100:.2f}% of an avalanche. Classification = {pred > threshold}")


Ten en cuenta que un umbral de 0.5 es solo un punto de partida que debe ajustarse en función de los datos que estamos tratando de clasificar.

# Rendimiento en el conjunto de pruebas

Ahora usemos nuestro conjunto de pruebas para realizar una evaluación rápida de cómo lo hizo el modelo. Por ahora, solo veremos cuántas veces predijimos correctamente si habría o no una avalancha.

In [ ]:
# Classify the mdel predictions using the threshold
predictions = model.predict(test) > threshold

# Compare the predictions to the actual outcomes in the dataset
accuracy = np.average(predictions == test.avalanche)

# Print the evaluation
print(f"The model correctly predicted outcomes {accuracy * 100:.2f}% of time.")

Las avalanchas son difíciles de predecir, pero lo estamos haciendo bien. Sin embargo, es difícil saber exactamente qué tipo de errores está cometiendo nuestro modelo. 